# Introduction: Testing Cyclical Encoding of Features for Machine Learning

Cyclical encoding of time-series features is typically done with time/date features that repeat periodically such as the hour of the day or day of the year. However, it is not a foregone conclusion this technique actually improves the performance of complex machine learning algorithms, such as a random forest. In this notebook, we will evaluate the efficacy of cyclical encoding of time-series features for predicting building energy data with two different models. The hypothesis is that cyclical encoding will improve the performance of the simple model, Linear Regression, but will not improve the performance of the model with higher capacity, the Random Forest Regression.

## Methods

__Feature Sets__
1. Standard = `["time_of_day", "day_of_year", "temperature"]`
2. Cyclical = `["sin_time_of_day", "cos_time_of_day", "sin_day_of_year", "cos_day_of_year", "temperature"]`

(We include temperature in both feature sets because when modeling time-series problems, we almost always have other features besides the date and time. Results may vary with the addition of features).

__Models__
1. Linear Regression = `LinearRegression(n_jobs=-1)`
2. Random Forest Regression = `RandomForestRegressor(n_estimators=25, max_depth=15, n_jobs=-1, random_state=100)`

(The hyperparameters for the random forest have proved to work well for use cases in our data science systems. Again, performance may vary with hyperparameter values).

We'll use a relatively simple test set-up. After creating the standard and cyclically encoded sets of features, we'll evaluate each set with the two models. Validation will consist of testing on a week of data at a time with all previous data used for training. We go through the dataset one week at a time, training on the historical data, and predicting for that selected week. This mimics what we actually do evaluating models for forecasting energy use at [Cortex Building Intel](https://cortexintel.com). The weekly validation is representative of a real-world situation, in which we'll have past training data and want to make a prediction of future energy consumption. By validating on past data, we can get an estimate of how the model will perform in the future. 

For our use case, we are primarily concerned with the performance of each set of features. We'll use a single metric, __mean absolute percentage error__ to assess our predictions. The benefits of MAPE are: relative comparisons, interpretability, and simplicity. The downsides are that it cna be sensitive to outliers and it's asymmetric because lower true values will result in greater percentage error for the same absolute error. There are always trade-offs to make in machine learning! Primarily we want to assess the difference in MAPE between the standard features and the cyclical features. Defining the difference in errors as standard mape - cyclical mape, a positive value indicates the cyclical features outperform the standard features. 


# Data 

We are using building energy data originally part of a DrivenData machine learning competition to forecast building energy use. You can download the original data from DrivenData here. I've cleaned up the building data and removed all features except for timestamp and temperature. The __objective__ is to predict the "energy" from the features.

In [1]:
import pandas as pd
import numpy as np
import glob

building_data_files = glob.glob('data/building*')
print(f'There are {len(building_data_files)} buildings to test.')

There are 40 buildings to test.


Let's take a look at one example dataset.

In [2]:
data = pd.read_csv(building_data_files[10], parse_dates=['timestamp'], index_col=0).set_index('timestamp')
data.head()

,temperature,energy
timestamp,,
2016-09-18 04:00:00,56.240300,1.682686
2016-09-18 04:15:00,56.087501,2.086212
2016-09-18 04:30:00,56.213232,1.687880
2016-09-18 04:45:00,56.400049,1.926518
2016-09-18 05:00:00,56.592497,1.922459


Nothing too extraordinary here.

In [3]:
import cufflinks as cf
from plotly.offline import plot

fig = data.iplot(y='energy', secondary_y='temperature', title='Temperature and Energy', asFigure=True)
plot(fig)

'file:///Users/williamkoehrsen/git/Data-Analysis/cyclical-features/temp-plot.html'

# Feature Engineering

To repeatedly create the same features, we'll make two scikit-learn transformers and then join them together in a pipeline. This is very simple code, but including it in a well-defined interface means we can use it over and over while confident we'll get the same operations applied to our data.

## Transformers

[Scikit-Learn transformers](http://scikit-learn.org/stable/data_transforms.html) give us a standard interface for applying operations to our data. Using transformers is efficient and allows you to build robust systems.

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin

class DateTimeFeatures(BaseEstimator, TransformerMixin):
    """
    Extract day of year and time of day features from a timestamp
    """
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Timestamps must be in index
        field = X.index
        X["time_of_day"] = field.hour + field.minute / 60
        X["day_of_year"] = field.dayofyear
        return X


class CyclicalDateTimeFeatures(BaseEstimator, TransformerMixin):
    """
    Make cyclically encoded day of year and time of day features
    """
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Apply formula for sin and cosine
        X["sin_time_of_day"], X["cos_time_of_day"] = _cyclical_encoding(
            X["time_of_day"], period=24
        )
        X["sin_day_of_year"], X["cos_day_of_year"] = _cyclical_encoding(
            X["day_of_year"], period=366
        )
        return X

def _cyclical_encoding(series, period):
    """
    Cyclical encoding of a series with a specified period
    """
    # Basic formula for sin/cosine equation
    base = 2 * np.pi * series / period
    return np.sin(base), np.cos(base)

## Pipeline

The pipeline is one of the most important features in sklearn. I highly recommend [reading up on the Pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) and using it throughout your data science projects.

In [5]:
from sklearn.pipeline import Pipeline

# Make a pipeline with the steps
transforms = Pipeline(
    steps=[
        # Must create the date/time features before encoding
        ("date_time_features", DateTimeFeatures()),
        ("cylical_date_time_features", CyclicalDateTimeFeatures()),
    ]
)

Now we can use our pipeline on any data set.

In [6]:
transformed_data = transforms.transform(data)
transformed_data.head()

,temperature,energy,time_of_day,day_of_year,sin_time_of_day,cos_time_of_day,sin_day_of_year,cos_day_of_year
timestamp,,,,,,,,
2016-09-18 04:00:00,56.240300,1.682686,4.00,262,0.866025,0.500000,-0.977064,-0.212947
2016-09-18 04:15:00,56.087501,2.086212,4.25,262,0.896873,0.442289,-0.977064,-0.212947
2016-09-18 04:30:00,56.213232,1.687880,4.50,262,0.923880,0.382683,-0.977064,-0.212947
2016-09-18 04:45:00,56.400049,1.926518,4.75,262,0.946930,0.321439,-0.977064,-0.212947
2016-09-18 05:00:00,56.592497,1.922459,5.00,262,0.965926,0.258819,-0.977064,-0.212947


In [7]:
data = pd.read_csv(building_data_files[20], parse_dates=['timestamp'], index_col=0).set_index('timestamp')
transformed_data = transforms.transform(data)
transformed_data.head()

,temperature,energy,time_of_day,day_of_year,sin_time_of_day,cos_time_of_day,sin_day_of_year,cos_day_of_year
timestamp,,,,,,,,
2014-01-01 05:15:00,36.736674,0.479060,5.25,1,0.980785,1.950903e-01,0.017166,0.999853
2014-01-01 05:30:00,36.682187,0.497991,5.50,1,0.991445,1.305262e-01,0.017166,0.999853
2014-01-01 05:45:00,36.679255,0.499107,5.75,1,0.997859,6.540313e-02,0.017166,0.999853
2014-01-01 06:00:00,36.699138,0.494427,6.00,1,1.000000,6.123234e-17,0.017166,0.999853
2014-01-01 06:15:00,36.587124,0.501165,6.25,1,0.997859,-6.540313e-02,0.017166,0.999853


In [8]:
fig = data.iplot(y=['sin_day_of_year', 'cos_day_of_year'], title="Cyclically Encoded Day of Year", asFigure=True)
plot(fig, filename='cylical_encoding_day_of_year.html')

'file:///Users/williamkoehrsen/git/Data-Analysis/cyclical-features/cylical_encoding_day_of_year.html'

In [9]:
fig = data.loc['2014-01-01': '2014-01-08'].iplot(y=['sin_time_of_day', 'cos_time_of_day'], title="Cyclically Encoded Time of Day", asFigure=True)
plot(fig, 'cyclical_encoding_time_of_day.html')

'file:///Users/williamkoehrsen/git/Data-Analysis/cyclical-features/temp-plot.html'

# Modeling

The modeling itself is relatively simple thanks to Scikit-Learn. We'll make a function that takes a set of data iterates through the sets of features (2) and the models (2). On each iteration, we apply the testing framework by stepping through the dataset one week at a time. We train on the all previous data and then test on the selected week recording the predictions along with the actual value.

This means that for each dataset, we are running 4 weekly validation testing sessions (2 x 2). Feel free to change the models or test using different features. You can also change the validation structure although we've found this to be an accurate indication of how well we can expect the model to perform on real data (forecasting for the future). 

If you were using this system in production, the most important aspect would be [writing unit tests](https://realpython.com/python-testing/)! These [should cover](https://docs.python-guide.org/writing/tests/) both the pipeline and the modeling. You are going to make mistakes in your code, that is guaranteed. Unit tests can help you find these mistakes before they cause unexpected errors in a machine learning system.

In [10]:
def run_weekly_validation(
    models,
    data,
    feature_sets=[
        ["time_of_day", "day_of_year", "temperature"],
        [
            "sin_time_of_day",
            "cos_time_of_day",
            "sin_day_of_year",
            "cos_day_of_year",
            "temperature",
        ],
    ],
):
    """
    Run the weekly validation modeling with models and feature sets on data
    
    :param models: list of sklearn models
    :param data: building energy dataframe
    :param feature_sets: list of lists of features
    
    :return results: dataframe of prediction results from each combination of model and feature set
    """

    all_predictions = []

    # Iterate through feature sets
    for feature_set in feature_sets:
        features = "standard" if "sin_time_of_day" not in feature_set else "cyclical"
        
        # Subset to features in data
        X = data[feature_set + ["energy"]].copy()

        # Iterate through models
        for model in models:
            
            model_name = model.__class__.__name__
            
            # Iterate through weeks in the dataset
            # Must group by string formatted week and year
            for (week, year), X_test in X.groupby(
                # [week, year] grouping
                [X.index.strftime("%U"), X.index.strftime("%Y")]
            ):
                # Subset to training data; all data before test set
                X_train = X[X.index < X_test.index.min()].copy()

                # Can not train or test on zero observations
                if len(X_train) == 0 or len(X_test) == 0:
                    continue

                # Targets
                y_train = X_train.pop("energy")
                y_test = X_test.pop("energy")

                # Fit and predict
                model.fit(X_train, y_train)
                predictions = model.predict(X_test)

                # Record predictions along with actual values, model, and feature set in a dataframe
                predictions = pd.DataFrame(
                    dict(
                        predicted=predictions,
                        actual=y_test,
                        model=model_name,
                        features=features,
                    ),
                    # Index is same as for testing data
                    index=X_test.index,
                )
                all_predictions.append(predictions)

    # Return dataframe of dataframes ordered by timestamp
    return (
        pd.concat(all_predictions)
        .reset_index()
        .sort_values(["model", "features", "timestamp"])
        .set_index("timestamp")
    )

We'll run one example first to make sure the results are as expected (in a real system, you'd want to have unit tests for this function and the pipeline.)

### Example Validation

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Create linear model and random forest model for regression
models = [
    LinearRegression(n_jobs=-1),
    RandomForestRegressor(n_estimators=25, max_depth=15, n_jobs=-1, random_state=100),
]

validation = run_weekly_validation(models, data)

/Users/williamkoehrsen/.local/share/virtualenvs/insight-engine-JU9-2WuC/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning:

internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.



In [12]:
validation.head()

,predicted,actual,model,features
timestamp,,,,
2014-01-05 00:00:00,0.424205,0.398189,LinearRegression,cyclical
2014-01-05 00:15:00,0.413624,0.397093,LinearRegression,cyclical
2014-01-05 00:30:00,0.408062,0.396569,LinearRegression,cyclical
2014-01-05 00:45:00,0.404033,0.400340,LinearRegression,cyclical
2014-01-05 01:00:00,0.400667,0.390858,LinearRegression,cyclical


In [13]:
validation.describe()

,predicted,actual
count,388268.000000,388268.000000
mean,0.243556,0.221177
std,0.127025,0.141697
min,-0.412401,0.014225
25%,0.157419,0.109462
50%,0.219884,0.175026
75%,0.301935,0.306944
max,1.389055,0.958976


## Validation for all Buildings

Finally, we can run the validation for all buildings. This may take a while and I've already included the results in the `validation_results` directory.

In [14]:
from tqdm import tqdm_notebook

def run_all_buildings(building_data_files):
    """
    Run weekly validation for all buildings
    
    :param building_data_files: list of filenames with building energy data csv files
    
    :return None: saves each file in `validation_results` for analysis
    """

    # Run validation for all buildings
    for building_file_name in tqdm_notebook(building_data_files, desc="Buildings"):

        building_data = pd.read_csv(
            building_file_name, parse_dates=["timestamp"]
        ).set_index("timestamp")
        # Create sets of features
        building_data = transforms.transform(building_data)

        # Run the validation and save the results
        building_validation = run_weekly_validation(models, building_data)

        # Save off results for analysis
        building_validation.to_csv(
            f"{building_file_name.replace('energy_data', 'validation_results').replace('data', 'validation_results')}"
        )

In [15]:
run_all_buildings(building_data_files)

# Analysis

Let's make sure we have all the datasets for analysis.

In [16]:
building_result_files = glob.glob('validation_results/*_validation_results.csv')
len(building_result_files)

40

We'll write our error metric as a function. There are some times when the actual value is 0 which results in infinite percentage error. To account for this, we replace the infinite values in the percentage error with a missing value and then ignore missing values in the mean absolute error calculation. The end outcome is a single float with the mean absolute error from the model's predictions.

In [17]:
def mean_absolute_percentage_error(actual, predicted):
    """
    Calculate the mean absolute percentage error of two arrays

    :param actual: true values as a pandas series
    :param predicted: estimated values as a pandas series

    :return mape: float of mean absolute percentage error (%)

    NOTE: We avoid getting an inf for the mape by replacing infinite values in the
    absolute_percentage_error with nan and then ignoring the missing values (np.nanmean)
    """
    ape_array = 100 * np.abs(predicted - actual) / actual
    return np.nanmean(ape_array.replace({np.inf: np.nan, -np.inf: np.nan}))

First, we'll test out a simple block of code to calculate the error metric for one building. This groups by the model and feature set and finds the mean absolute error of the predictions.

In [18]:
building_result_file = building_result_files[20]

validation_results = pd.read_csv(building_result_file)

building_metrics = (
    validation_results.groupby(
        [validation_results["features"], validation_results["model"]]
    )
    .apply(lambda x: mean_absolute_percentage_error(x["actual"], x["predicted"]))
    .to_frame()
    .rename(columns={0: "mean_absolute_percentage_error"})
)

building_metrics

mean_absolute_percentage_error
features model                                                
cyclical LinearRegression                            36.098788
         RandomForestRegressor                       18.749387
standard LinearRegression                            42.860637
         RandomForestRegressor                       18.747120

That works, so now we'll calculate the metrics for all buildings. The above code can go in a function.

In [19]:
def calculate_metrics(building_result_file):
    """
    Calculate mean_absolute_percentage_error for a building from a predictions file
    
    :param building_result_file: location of validation results data
    
    :return building_metrics: dataframe of metrics for building
    """

    # Read in the results
    validation_results = pd.read_csv(building_result_file)

    # Groupby the feature set and model
    building_metrics = (
        validation_results.groupby(
            ["features", "model"]
        )
        # Calculate the error metric
        .apply(lambda x: mean_absolute_percentage_error(x["actual"], x["predicted"]))
        # Format the dataframe
        .to_frame()
        .reset_index()
        .rename(columns={0: "mean_absolute_percentage_error"})
    )
    # Add the building id
    building_metrics["building_id"] = building_result_file.split("_")[2]
    return building_metrics

Now, we apply that function to all building results.

In [20]:
# Create a dataframe of all metrcs
all_metrics = pd.concat(
    [calculate_metrics(br_file) for br_file in building_result_files]
)
all_metrics.to_csv("validation_results/all_metrics.csv")

In [21]:
all_metrics.head()
all_metrics.describe()

,features,model,mean_absolute_percentage_error,building_id
0,cyclical,LinearRegression,17.909297,24
1,cyclical,RandomForestRegressor,12.280728,24
2,standard,LinearRegression,20.352490,24
3,standard,RandomForestRegressor,12.487683,24
0,cyclical,LinearRegression,29.121338,15


,mean_absolute_percentage_error
count,160.000000
mean,63.880972
std,83.319291
min,9.202885
25%,22.299167
50%,32.141760
75%,65.297555
max,451.598475


# Results

We can now answer our main question: does cyclical encoding of date/time features benefit machine learning models? If so, we expect to see a decrease in the model error with the cyclical features.

In [22]:
all_metrics.groupby(['features', 'model']).describe()

mean_absolute_percentage_error             \
                                                        count       mean   
features model                                                             
cyclical LinearRegression                                40.0  68.862609   
         RandomForestRegressor                           40.0  55.002952   
standard LinearRegression                                40.0  76.941931   
         RandomForestRegressor                           40.0  54.716396   

                                                                            \
                                      std        min        25%        50%   
features model                                                               
cyclical LinearRegression       85.376630  10.377362  27.981233  34.571894   
         RandomForestRegressor  77.700614   9.202885  19.229364  23.832743   
standard LinearRegression       94.564915  10.284414  29.178142  41.586746   
         RandomForestRegressor  75.272497   9.321820  19.344407  23.813667   

                                                       
                                      75%         max  
features model                                         
cyclical LinearRegression       67.116970  438.634348  
         RandomForestRegressor  48.580763  413.009500  
standard LinearRegression       68.941804  451.598475  
         RandomForestRegressor  48.092324  386.051141

To find the answer, we group by the building and model and calculate the difference in error between the sets of features.

In [23]:
def difference_between_features(metrics):
    """
    Calculate the difference in mean absolute error between two sets of features.
    """
    return float(
        metrics.loc[
            metrics["features"] == "standard", "mean_absolute_percentage_error"
        ].values
        - metrics.loc[
            metrics["features"] == "cyclical", "mean_absolute_percentage_error"
        ].values
    )

# Groupby building and model
feature_differences = (
    all_metrics.groupby(["building_id", "model"])
    # Calculate difference
    .apply(lambda x: difference_between_features(x))
    # Format as dataframe
    .to_frame()
    .reset_index()
    .rename(columns={0: "difference_in_error"})
)

feature_differences.groupby('model').describe()

difference_in_error                                  \
                                    count      mean        std        min   
model                                                                       
LinearRegression                     40.0  8.079322  14.717504  -7.106086   
RandomForestRegressor                40.0 -0.286556   5.676348 -26.958359   

                                                                
                            25%       50%       75%        max  
model                                                           
LinearRegression       0.422033  4.954048  8.216279  64.505072  
RandomForestRegressor -0.358428  0.096175  0.255632  16.377598

The initial hypothesis holds: __Cyclical encoding of features benefits a simple linear regression, but not a complex model such as Random Forest Regression.__

# Visualizations

We've answered our main question, but we can still make some visualizations. We'll keep using the interactive [plotly library](https://plot.ly/python/) with its newest addition, [plotly express](https://medium.com/@plotlygraphs/introducing-plotly-express-808df010143d)

In [24]:
from plotly.offline import plot
import plotly_express as px

In [25]:
def graph_results(all_metrics, feature_differences):
    # Make a bar plot of all the errors for each model and set of features
    all_scores_fig = px.bar(
        all_metrics,
        x="building_id",
        y="mean_absolute_percentage_error",
        color="features",
        barmode="group",
        facet_row="model",
        title="Comparison of Models and Errors",
        template="plotly_white",
        height=1400,
    )
    # Make a bar plot of the difference in errors from cyclical encoding
    diff_fig = px.bar(
        feature_differences,
        x="building_id",
        y="difference_in_error",
        facet_row="model",
        height=1400,
        template="plotly_white",
        title="Performance Gain from Cyclical Encoding of Features",
    )
    return all_scores_fig, diff_fig


all_scores_fig, diff_fig = graph_results(all_metrics, feature_differences)

In [26]:
plot(all_scores_fig, filename="validation_results/all_scores.html")
plot(diff_fig, filename='validation_results/diff_in_scores.html')

'validation_results/all_scores.html'

'validation_results/diff_in_scores.html'

The graphs clearly show our conclusion: the cyclically encoding has lowered the error for the linear regression but not for the random forest. Feel free to explore the data and change the testing setup, model hyperparameters, feature sets, data sets to see if this result holds.

# Conclusion

The main takeaway is that __cyclical feature encoding is not required for complex models with a high capacity.__ These models are able to learn the relationships between the day of year, time of day and the target without cyclical encoding. The feature transformation does not hurt the more complex models, but it adds unnecessary steps to a machine learning pipeline without adding benefits. 

Granted, this result may only hold for building energy data without many other features and I think it'd be great if others applied a similar testing procedure to different datasets. Currently, the only way to figure out if something is a good idea in machine learning is to get some data and try it out, a lesson we learned with this feature testing project.